In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import requests
from IPython.display import display, Markdown

# Load the dataset and initialize required columns
try:
    file_path = "C:/Users/96655/Downloads/SWE485-Project-Group9/Dataset/cleaned_dataset.csv"
    df = pd.read_csv(file_path)
    df["Original_Area"] = df["Area"]
    df["Original_Price"] = df["Price"]
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    raise

# Preprocess the data
features = ["Area", "Price"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

# Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Function to make recommendations based on KMeans clustering
def recommend_properties_kmeans(price_input, area_input, top_n=5):
    input_df = pd.DataFrame([[area_input, price_input]], columns=["Area", "Price"])
    input_scaled = scaler.transform(input_df)
    input_cluster = kmeans.predict(input_scaled)[0]

    cluster_data = df[df["Cluster"] == input_cluster].copy()
    cluster_scaled = scaler.transform(cluster_data[["Area", "Price"]])
    distances = euclidean_distances(input_scaled, cluster_scaled)[0]

    cluster_data["Distance"] = distances
    recommended = cluster_data.sort_values("Distance").head(top_n)

    recommended = recommended.rename(columns={
        "Property Type": "Property Type",
        "Location": "Location",
        "District": "District",
        "Bedrooms": "Bedrooms",
        "Bathrooms": "Bathrooms",
        "Original_Area": "Area",
        "Original_Price": "Price",
        "Agency_Name": "Agency"
    })

    headers = ["Property Type", "Location", "District", "Bedrooms", "Bathrooms", "Area", "Price", "Agency"]
    return recommended[headers]

def template_informative(properties):
    explanation = (
        "You are a helpful real estate assistant.\n"
        "Respond in English.\n\n"
        "However, you must display all property attributes using the following Arabic labels (do NOT translate them):\n"
        "نوع العقار، الموقع، الحي، الغرف، دورات المياه، المساحة، السعر، الوكالة\n\n"
        "You must also keep all property values exactly as they are, including Arabic values such as: شقة، مكتب الاحمدي للعقارات، شرق - قرطبة.\n"
        "Do not translate or transliterate any Arabic value.\n\n"
        "Present each property in a bullet point format like this:\n"
        "- نوع العقار: (value)\n"
        "- الموقع: (value)\n"
        "- الحي: (value)\n"
        "- الغرف: (value)\n"
        "- دورات المياه: (value)\n"
        "- المساحة: (value) متر مربع\n"
        "- السعر: (value) ريال\n"
        "- الوكالة: (value)\n\n"
        "After listing each property, provide a short paragraph in English describing the property in a professional and informative tone.\n"
        "Focus on who it is suitable for, and highlight any standout features.\n\n"
        "Here are the suggested properties:\n\n"
    )

    for i, prop in enumerate(properties):
        explanation += (
            f"{i+1}.\n"
            f"- نوع العقار: {prop['Property Type']}\n"
            f"- الموقع: {prop['Location']}\n"
            f"- الحي: {prop['District']}\n"
            f"- الغرف: {prop['Bedrooms']}\n"
            f"- دورات المياه: {prop['Bathrooms']}\n"
            f"- المساحة: {prop['Area']} متر مربع\n"
            f"- السعر: {prop['Price']} ريال\n"
            f"- الوكالة: {prop['Agency']}\n\n"
        )

    explanation += "These properties were selected using the KMeans clustering algorithm."
    return explanation




def template_friendly_descriptive(properties):
    response = (
        "You are a friendly real estate assistant. Respond in English, but you MUST use these Arabic field labels only:\n"
        "نوع العقار، الموقع، الحي، الغرف، دورات المياه، المساحة، السعر، الوكالة\n"
        "Also, do NOT translate or transliterate any Arabic values. Keep them exactly as they are.\n"
        "You must also keep all property values exactly as they are, including Arabic values such as: شقة، مكتب الاحمدي للعقارات، شرق - قرطبة.\n"
        "Do not translate or transliterate any Arabic value.\n"
        "Present each property in a bullet point format like this:\n"
        "- نوع العقار: (value)\n"
        "- الموقع: (value)\n"
        "- الحي: (value)\n"
        "- الغرف: (value)\n"
        "- دورات المياه: (value)\n"
        "- المساحة: (value) متر مربع\n"
        "- السعر: (value) ريال\n"
        "- الوكالة: (value)\n\n"
        "After listing each property, provide a short description of the property in a friendly, approachable tone.\n"
        "Focus on making it engaging and conversational. Let the assistant generate the description freely and in a friendly manner, without following any specific logic or rules.\n\n"
        "Here are the suggested properties:\n\n"
    )

    for i, prop in enumerate(properties):
        response += (
            f"{i+1}.\n"
            f"- نوع العقار: {prop['Property Type']}\n"
            f"- الموقع: {prop['Location']}\n"
            f"- الحي: {prop['District']}\n"
            f"- الغرف: {prop['Bedrooms']}\n"
            f"- دورات المياه: {prop['Bathrooms']}\n"
            f"- المساحة: {prop['Area']} متر مربع\n"
            f"- السعر: {prop['Price']} ريال\n"
            f"- الوكالة: {prop['Agency']}\n\n"
        )

    return response

    
# Call the LLaMA Chat API
def call_llama(prompt, model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.6, top_p=0.85):
    TOGETHER_API_KEY = "388185048d5c7f46d5b23dafd68068a565ab9a463bba9682a50a993772c4333d"

    if not TOGETHER_API_KEY:
        raise ValueError("API key not found.")

    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful real estate assistant."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens":  7000,
        "temperature": temperature,
        "top_p": top_p,
        "repetition_penalty": 1.1,
    }

    try:
        
        response = requests.post(
            "https://api.together.xyz/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status()

        result = response.json()
      

        return result["choices"][0]["message"]["content"]
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")

# Ensure all columns are unique
def make_columns_unique(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [
            dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))
        ]
    df.columns = cols
    return df

# Generate recommendations and display results
recommended = recommend_properties_kmeans(price_input=6000, area_input=300)
if recommended.empty:
    print("No properties found based on your input.")
else:
    recommended_unique = make_columns_unique(recommended)
    property_dicts = recommended_unique.to_dict(orient="records")

    prompt1 = template_informative(property_dicts)
    prompt2 = template_friendly_descriptive(property_dicts)

    try:
        response1 = call_llama(prompt1) if property_dicts else "No properties found"
        response2 = call_llama(prompt2) if property_dicts else "No properties found"

        display(Markdown("### Template 1: Informative Guide"))
        display(Markdown(response1.strip()))

        display(Markdown("### Template 2: Casual Guide"))
        display(Markdown(response2.strip()))
    except Exception as e:
        print("Failed to call API:", str(e))


### Template 1: Informative Guide

Here are the properties listed with their attributes:

1.
- نوع العقار: شقة
- الموقع: شرق
- الحي: قرطبة
- الغرف: 3
- دورات المياه: 3
- المساحة: 360 متر مربع
- السعر: 65000 ريال
- الوكالة: مكتب عبدالله بن فيحان بن محمد العتيبي للعقارات

This apartment is located in the east part of the city, in the neighborhood of Qurtaba. It has three bedrooms, three bathrooms, and a spacious living area of 360 square meters. The price of this apartment is quite affordable at 65,000 riyals, making it an excellent option for small families looking for a new home.

2.
- نوع العقار: دور
- الموقع: جنوب
- الحي: الدار البيضاء
- الغرف: 4
- دورات المياه: 3
- المساحة: 184 متر مربع
- السعر: 1400 ريال
- الوكالة: شركة حلول دارك العقارية

This house is situated in the southern part of the city, in the Dar Al Baydaa neighborhood. With four bedrooms and three bathrooms, this house is perfect for larger families. It has a compact living space of 184 square meters, but its affordable price of only 1,400 riyals makes up for it. This property is managed by Darak Real Estate Solutions.

3.
- نوع العقار: شقة
- الموقع: جنوب
- الحي: الدار البيضاء
- الغرف: 3
- دورات المياه: 2
- المساحة: 219 متر مربع
- السعر: 350000 ريال
- الوكالة: مكتب الاحمدي للعقارات

This apartment is located in the south part of the city, in the Dar Al Baydaa neighborhood. It has three bedrooms and two bathrooms, with a total living space of 219 square meters. While the price of 350,000 riyals may be high for some, the property's location and size make it an attractive option for those looking for a spacious home.

4.
- نوع العقار: شقة
- الموقع: جنوب
- الحي: الدار البيضاء
- الغرف: 2
- دورات المياه: 3
- المساحة: 203 متر مربع
- السعر: 400000 ريال
- الوكالة: مكتب الاحمدي للعقارات

This apartment is situated in the southern part of the city, in the Dar Al Baydaa neighborhood. It has two bedrooms and three bathrooms, with a living space of 203 square meters. Although the price of 400,000 riyals may seem steep, the apartment's ample living space and location make it an ideal choice for couples or small families seeking a new home.

5.
- نوع العقار: شقة
- الموقع: غرب
- الحي: الزهرة
- الغرف: 4
- دورات المياه: 3
- المساحة: 250 متر مربع
- السعر: 550000 ريال
- الوكالة: شركة نجمة التميز للعقارات

This apartment is located in the west part of the city, in the Zahra neighborhood. It boasts four bedrooms and three bathrooms, with a spacious living area of 250 square meters. Despite the higher price tag of 550,000 riyals, this property offers plenty of room for a large family and is managed by Najm Al Tamayz Real Estate Company.

### Template 2: Casual Guide

* **Property 1:**
	+ Type of property: Apartment
	+ Location: East

In [1]:
comparison_markdown = '''
### Comparison of Prompt Templates

We implemented two prompt templates to enhance the real estate recommendation experience using Generative AI.

**Template 1** presents the properties in a structured, clear format. It lists each attribute consistently and includes a professional explanation highlighting who the property is suitable for and what makes it appealing. This template is ideal for users who value concise and organized information that helps with practical decision-making.

**Template 2** takes a more conversational and friendly approach. While it displays the same property attributes, it follows up with a natural, descriptive explanation for each property, giving a more human touch. The descriptions feel more personalized and easier to relate to, which can enhance user engagement.

---

### Chosen Template

We selected **Template 2** as the more effective option. It allows the assistant to present the listings in a way that feels more engaging and supportive, which aligns better with what most users look for when browsing properties—clear details paired with a relatable explanation that helps them imagine living in the space.
'''

from IPython.display import Markdown, display

display(Markdown(comparison_markdown))



### Comparison of Prompt Templates

We implemented two prompt templates to enhance the real estate recommendation experience using Generative AI.

**Template 1** presents the properties in a structured, clear format. It lists each attribute consistently and includes a professional explanation highlighting who the property is suitable for and what makes it appealing. This template is ideal for users who value concise and organized information that helps with practical decision-making.

**Template 2** takes a more conversational and friendly approach. While it displays the same property attributes, it follows up with a natural, descriptive explanation for each property, giving a more human touch. The descriptions feel more personalized and easier to relate to, which can enhance user engagement.

---

### Chosen Template

We selected **Template 2** as the more effective option. It allows the assistant to present the listings in a way that feels more engaging and supportive, which aligns better with what most users look for when browsing properties—clear details paired with a relatable explanation that helps them imagine living in the space.
